# 💻 WAE Chatbot Workshop

## Installing Required Libraries
We will use:

Gradio for building simple user interfaces.

Mistral AI for accessing the language model.


In [ ]:
!pip install mistralai
!pip install gradio

from IPython.display import clear_output
clear_output() # Clear previous output for a cleaner notebook

Our API is currently available through [La Plateforme](https://console.mistral.ai/). You need to activate payments on your account to enable your API keys. After activation, you can start using the `chat` endpoint to interact with models.

##  Setting up the Mistral Model

You will need to add your API Key from:  https://console.mistral.ai/api-keys/


You can explore all available models here: https://docs.mistral.ai/getting-started/models/models_overview/



In [ ]:
from mistralai import Mistral

api_key = "YOUR_API_KEY_HERE" # 👉 Replace with your own API Key
model = "mistral-large-2402"  # You can change the model if you want

client = Mistral(api_key=api_key)

# Building a Simple Chatbot using Mistral Model

In [ ]:
def chat_with_mistral(message, history):
    # Build the conversation context by combining the existing history with the new user message.
    messages = history + [{"role": "user", "content": message}]

    chat_response = client.chat.complete(
        model=model,
        messages=messages # Content from the user input
    )
    response = chat_response.choices[0].message.content

    formatted_response = {"role": "assistant", "content": response}

    # Return only the new assistant message.
    return formatted_response

# Creating a Web Interface using Gradio

In [ ]:
import gradio as gr

iface = gr.ChatInterface(
    fn=chat_with_mistral, # Function to handle chat messages
    type="messages", # Set the input/output format to handle full chat messages
    title="Mistral AI Chatbot",
    description="Chat with the Mistral AI large language model.",
    theme="glass", # Visual theme
)

iface.launch(
    debug=True, # Show detailed error messages if something goes wrong
    share=True, # Create a public link to share your app with others
)

## 🛠 Exercise: E-commerce Store Chatbot
You are building a chatbot for an online store that sells electronics (phones, laptops, and accessories).

The chatbot should answer any questions related to the store’s products and services.

If users ask about topics not related to the store or its offerings, the chatbot should politely inform them that it can only assist with store-related inquiries.

In [ ]:
# 1. Define the system prompt here
system_prompt = {???}  # 👉 Here you should put 2 elements..

# 2. Create the chat function
def chat_with_store_bot(user_message, history):
    ???

# 3. Create the interface
store_iface = gr.ChatInterface(???) # 👉 Only two parameters are necessary: 'fn' (function name) and 'type' (set it to "messages").

# 4. Launch the interface
store_iface.launch()

### 💡 Hint

<details>
<summary>Click here to see the hint</summary>

When writing your system prompt, follow this structure:

```python
{"role": "system", "content": "Your special instruction goes here."}


&nbsp;  
&nbsp;  
&nbsp;

# Building a Chatbot using Hugging Face Transformers

## 🤖 About the microsoft/DialoGPT-medium Model

In this section, we introduce the **`microsoft/DialoGPT-medium`** model — a small open-domain chatbot developed by Microsoft and trained on casual Reddit conversations.  
Unlike the **`mistral-large-2402`** model we used earlier, which is a large instruction-following language model designed for a wide range of complex tasks, **DialoGPT-medium** is much smaller and specialized mainly for casual, free-flowing conversations without strict understanding or task execution.  
As a result, you may notice that **DialoGPT's responses are often less focused, more random, and sometimes humorous**, compared to the more structured and instruction-driven answers generated by Mistral.  
This will give you a practical sense of how **different training objectives and model sizes** impact the quality and behavior of chatbots.  

Chatbot fine-tuning will not be covered in this workshop, as it is a challenging task that typically requires deep expertise, significant resources, and specialized datasets.

In [ ]:
!pip install transformers
!pip install torch

from IPython.display import clear_output
clear_output()

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "microsoft/DialoGPT-medium"  # Choose a chatbot model from Hugging Face
tokenizer = AutoTokenizer.from_pretrained(model_name) # tokenizer: converts text into tokens that the model can understand
model = AutoModelForCausalLM.from_pretrained(model_name) # model: generates text responses based on the input tokens

In [ ]:
# Define the chat function
def chat_with_dialoGPT(user_input, chat_history_ids=None):
    # Encode the user input and add end-of-sentence token
    new_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')

    # If there is history, append the new input to history; else, just use the new input
    if chat_history_ids is not None:
        input_ids = torch.cat([chat_history_ids, new_input_ids], dim=-1)
    else:
        input_ids = new_input_ids

    # Generate the model's response
    chat_history_ids = model.generate(
        input_ids,
        max_length=1000,               # maximum number of tokens for input + output
        pad_token_id=tokenizer.eos_token_id,  # Padding token (using EOS token, which marks end of sentence)
        do_sample=True,                # Enable sampling to create fun, more random responses
        top_p=0.92,                    # Nucleus sampling (only pick from top 92% most likely next words)
        temperature=0.75,              # Control randomness (slightly creative), higher values = more creative
        repetition_penalty=1.1,         # Slightly discourage the model from repeating itself
        no_repeat_ngram_size=3          # Prevent repeating any sequence of 3 words to make conversation sound more natural
    )

    # Decode the model's response
    bot_response = tokenizer.decode(chat_history_ids[:, input_ids.shape[-1]:][0], skip_special_tokens=True)

    return bot_response, chat_history_ids

In [ ]:
# Initialize conversation history (for DialoGPT, it's token IDs history)
chat_history_ids = None

while True:
    user_input = input("You: ")
    if user_input.lower() in ["quit", "q", "exit"]:
        print("Chat ended.")
        break

    response, chat_history_ids = chat_with_dialoGPT(user_input, chat_history_ids)
    print(f"Chatbot: {response}")

> 💬 **Note:**  
> Of course, you can use **Gradio** to build a web-based chatbot interface as we did in the previous example. However, for the purpose of showing different methods, we are using the simple input/output approach.

## 🛠 Exercise: Try a Different Hugging Face Model!

You can find many pre-trained models on the [Hugging Face Model Hub](https://huggingface.co/models).
To try a different model, follow these steps:
1. Visit the Hugging Face Models page.
2. Search for a model that matches your goal (for example, chatbot, text generation, question answering).
3. Copy the model name exactly as shown (for example, `microsoft/DialoGPT-medium`).
4. Replace the old model name inside your `AutoTokenizer.from_pretrained()` and `AutoModelForCausalLM.from_pretrained()` code.
5. Test your chatbot again to see how different models behave!

In [ ]:
model_name = ???
tokenizer = ???
model = ???

def chat_with_your_model(user_input, chat_history_ids=None):
  ???

In [ ]:
# Initialize conversation history (for DialoGPT, it's token IDs history)
chat_history_ids = None

while True:
    user_input = input("You: ")
    if user_input.lower() in ["quit", "q", "exit"]:
        print("Chat ended.")
        break

    response, chat_history_ids = ???
    print(f"Chatbot: {response}")

# 🎉 Congratulations

Congratulations on completing the workshop!

Today, you learned how to build chatbots using both **Mistral AI** and **Hugging Face Transformers**.

Keep experimenting, explore new models, and never stop learning. 🚀  
The world of AI is full of opportunities — this is just the beginning!